In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
from torch.autograd.functional import jacobian
from operators import Δ, div, D, mdotb, bdotm, mdotm, bdotb
from torch.nn.functional import relu, max_pool2d, avg_pool2d, dropout, dropout2d, interpolate

In [2]:
class Complex_ReLU(nn.Module):
    def forward(self, input):
        return relu(input.real).type(torch.complex64)+1j*relu(input.imag).type(torch.complex64)

In [ ]:
complex_net = nn.Sequential(
            nn.Linear(1, 4).to(torch.cfloat),
            Complex_ReLU(),
            nn.Linear(4, 1).to(torch.cfloat)
        )
tensor = torch.randn(4, 2).requires_grad_()
tensor
torch.view_as_complex(tensor).unsqueeze(1)

In [ ]:
u = complex_net(torch.view_as_complex(tensor).unsqueeze(1))
u = torch.exp(torch.abs(1j*u))

In [ ]:
div(u, tensor)

In [ ]:
var = torch.tensor([[1.0, 1.0],
                   [4.0, 3.0]], requires_grad=True)
t = torch.tensor([[2.0], 
                  [3.0]], requires_grad=True)

var_2 = torch.tensor([[1.0],
                   [3.0]], requires_grad=True)
t_2 = torch.tensor([[1.0], 
                  [2.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0].unsqueeze(1), 3) + xt[:,-1].unsqueeze(1)

In [ ]:
spatial_net = nn.Sequential(
            nn.Linear(2, 4),
            nn.ELU(),
            nn.Linear(4, 4),
            nn.ELU(),
            nn.Linear(4, 1)
        )

In [ ]:
out = u(var, t)
out_2 = u(var_2, t_2)

In [ ]:
torch.sum(jacobian(spatial_net, var), dim=0)

In [ ]:
out = spatial_net(var)
div(out, var)

In [ ]:
print(div(out.unsqueeze(1), t))
print(div(out_2.unsqueeze(1), t_2))

In [ ]:
print(div(out.unsqueeze(1), var))
print(div(out_2.unsqueeze(1), var_2))

In [ ]:
print(Δ(out.unsqueeze(1), var))
print(Δ(out_2.unsqueeze(1), var_2))

In [ ]:
loss = torch.square(div(out, t) - Δ(out, var)).mean()

In [ ]:
loss.backward()

In [3]:
var_1 = torch.tensor([[1.0],
                      [4.0]], requires_grad=True)
var_2 = torch.tensor([[2.0],
                      [8.0]], requires_grad=True)
var = [var_1, var_2]
C = torch.tensor([[2.0, 4.0],
                  [1.0, 3.0]], requires_grad=True)
def u(x, t):
    xt = torch.cat((x, t), dim=1)
    return (1/6)*torch.pow(xt[:, 0].unsqueeze(1), 3) + torch.pow(xt[:,-1].unsqueeze(1), 2)

In [4]:
us = u(var_1, var_2)
print(us)

tensor([[ 4.1667],
        [74.6667]], grad_fn=<AddBackward0>)


In [10]:
div(us, var_1) + div(us, var_2) + div(div(us, var_1), var_1) + div(div(us, var_2), var_2)

tensor([[ 7.5000],
        [30.0000]], grad_fn=<AddBackward0>)

In [15]:
D(us, var) + (D(us, var))**2

tensor([[  0.7500,  20.0000],
        [ 72.0000, 272.0000]], grad_fn=<AddBackward0>)

In [ ]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)

In [ ]:
def m_func(x, y):
    if isinstance(x, list):
        x = torch.cat(x, dim=-1)   
        return torch.einsum("bi, ij -> bj", x, y)
    y = torch.cat(y, dim=-1)
    return torch.einsum("ij, bj -> bi", x, y)
        
def v_func(x, y):
    if isinstance(x, list):
        x = torch.cat(x, dim=-1)
    if isinstance(y, list):
        y = torch.cat(y, dim=-1)
    
    return torch.einsum("bi, bi -> b", x, y).unsqueeze(1)
    
        

In [64]:
var_1 = torch.tensor([[1.0],
                     [2.0]], requires_grad=True)
var_2 = torch.tensor([[1.0],
                     [3.0]], requires_grad=True)
var_3 = torch.tensor([[2.0],
                     [4.0]], requires_grad=True)
var = [var_1, var_2, var_3]
C = torch.tensor([[2.0, 4.0],
                  [1.0, 3.0]], requires_grad=True)
def us(*v):
    return v[1]**3*var[2] + v[1]**3*var[0]**3 +v[2]**3*var[0]**2

In [65]:
u = us(*var)
u_grad = grad(outputs=u, inputs=var, create_graph=True, grad_outputs=torch.ones_like(u))
u_grad
u_grad = torch.cat(u_grad, dim=-1)

# Computes Hessian (Careful)

In [45]:
print(u_grad)
for i in range(u_grad.size(0)):
    for j in range(u_grad.size(1)):
        print(torch.autograd.grad(u_grad[i][j], var, retain_graph=True))

tensor([[19.,  9., 13.],
        [19.,  9., 13.]], grad_fn=<CatBackward>)
(tensor([[22.],
        [ 0.]]), tensor([[9.],
        [0.]]), tensor([[24.],
        [ 0.]]))
(tensor([[9.],
        [0.]]), tensor([[18.],
        [ 0.]]), tensor([[3.],
        [0.]]))
(tensor([[24.],
        [ 0.]]), tensor([[3.],
        [0.]]), tensor([[12.],
        [ 0.]]))
(tensor([[ 0.],
        [22.]]), tensor([[0.],
        [9.]]), tensor([[ 0.],
        [24.]]))
(tensor([[0.],
        [9.]]), tensor([[ 0.],
        [18.]]), tensor([[0.],
        [3.]]))
(tensor([[ 0.],
        [24.]]), tensor([[0.],
        [3.]]), tensor([[ 0.],
        [12.]]))


In [46]:
for i in range(u_grad.size(1)):
    print(grad(u_grad[:, i], var, retain_graph=True, grad_outputs=torch.ones_like(u_grad[:, i])))

(tensor([[22.],
        [22.]]), tensor([[9.],
        [9.]]), tensor([[24.],
        [24.]]))
(tensor([[9.],
        [9.]]), tensor([[18.],
        [18.]]), tensor([[3.],
        [3.]]))
(tensor([[24.],
        [24.]]), tensor([[3.],
        [3.]]), tensor([[12.],
        [12.]]))


In [42]:
print(grad(u_grad[:, 0], var, retain_graph=True, grad_outputs=torch.ones_like(u_grad[:, 0])))

(tensor([[22.]]), tensor([[9.]]), tensor([[24.]]))


In [24]:
u_grad.repeat(1, 3)

tensor([[19.,  9., 13., 19.,  9., 13., 19.,  9., 13.]],
       grad_fn=<RepeatBackward>)

In [ ]:
(C |mdotb| torch.cat(var[:2], dim=-1))

In [ ]:
torch.cat((var_1, var_2, var_1), dim=-1)

In [32]:
u_grad.reshape(-1, 1)


tensor([[19.],
        [ 9.],
        [13.]], grad_fn=<ViewBackward>)

In [68]:
def hessian(u, var):
    hessian_rows = []
    u_grads = list(grad(outputs=u, inputs=var, create_graph=True, grad_outputs=torch.ones_like(u)))
    for u_grad in u_grads:
        hessian_rows.append(torch.cat(grad(u_grad, var, create_graph=True, grad_outputs=torch.ones_like(u_grad)), dim=-1).unsqueeze(-1))
    return torch.cat(hessian_rows, dim=-1)

In [69]:
hessian(u, var)

tensor([[[ 22.,   9.,  24.],
         [  9.,  18.,   3.],
         [ 24.,   3.,  12.]],

        [[452., 324., 192.],
         [324., 216.,  27.],
         [192.,  27.,  96.]]], grad_fn=<CatBackward>)